In [1]:
#In Python, ast stands for Abstract Syntax Trees. 
#Abstract Syntax Trees (ASTs) represent the hierarchical structure of the source code 
#without including specific details about individual characters or formatting..
#Iam adding this too
import ast
#I am adding this to test
source_code = "print('Hello, world!')"
parsed_ast = ast.parse(source_code)
#adding the 3rd change

In [6]:
import pandas as pd


In [8]:
df = pd.read_csv('file_path\\unicorns.csv')

In [9]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN


In [153]:
investor_list = "[1, 2, 3, 4]"

try:
    # Safely evaluate the string using ast.literal_eval
    result = ast.literal_eval(investor_list)
    print(result)
except (SyntaxError, ValueError) as e:
    print(f"Error: {e}")
    

[1, 2, 3, 4]


In [34]:
def summary(company,crunchbase_url,city,country,industry,investor_list):
    investors = 'The investors of this company are'
    
    for i in ast.literal_eval(investor_list):
        investors +=f' {i}, '
        
    text = f'{company} has HQ in {city} in {country} and is in the field of {industry}. {investors}. More info on {crunchbase_url}'  
    return text

In [35]:
df['summary'] = df.apply(lambda df: summary(df['Company'],df['Crunchbase Url'],df['City'],df['Country'],df['Industry'],df['Investors']),axis=1)

In [36]:
df['summary'][0]

'Esusu has HQ in New York in United States and is in the field of Fintech. The investors of this company are Next Play Ventures,  Zeal Capital Partners,  SoftBank Group, . More info on https://www.cbinsights.com/company/esusu'

In [38]:
import tiktoken

In [41]:
def num_tokens(string, encodeing_model):
    encoding = tiktoken.get_encoding(encodeing_model)
    num_tokens = len(encoding.encode(string))
    
    return num_tokens

In [42]:
num_tokens(df['summary'][0], 'cl100k_base')

55

In [44]:
#check the number of words to comapre with tokens
len(df['summary'][0].split())

35

In [45]:
df['token_count'] = df['summary'].apply(lambda x: num_tokens(x,'cl100k_base' ))

In [47]:
#total num of tokens in summary col
df['token_count'].sum()

66824

In [48]:
total_price = df['token_count'].sum()/1000 * 0.0001

In [49]:
#in USD
total_price

0.0066824

In [52]:
#checking the length of each row to fit in max token size for ADA model
df[(df['token_count'] > 8191)]

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count


In [ ]:
import openai
from openai import OpenAI
import os

os.environ['OPENAI_API_KEY'] = '####'
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=openai.api_key)

In [57]:
def get_embedding(text):
    response = client.embeddings.create(
    input=text,
    model="text-embedding-ada-002"
)
    return (response.data[0].embedding) 

In [66]:
emb_list = get_embedding(df['summary'][0])

In [67]:
len(emb_list)

1536

In [68]:
import numpy as np

In [70]:
np.array(emb_list).shape

(1536,)

In [72]:
df['embedding'] = df['summary'].apply(lambda x: get_embedding(x))

In [80]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding,embedding_to_arr
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has HQ in New York in United States and ...,55,"[0.012066167779266834, -0.015717506408691406, ...","[0.012066167779266834, -0.015717506408691406, ..."
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has HQ in New York in United States...,57,"[0.00999634712934494, 0.013905090279877186, -0...","[0.00999634712934494, 0.013905090279877186, -0..."
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has HQ in Palo Alto in United States and...,54,"[0.0030284523963928223, -0.0391857773065567, 0...","[0.0030284523963928223, -0.0391857773065567, 0..."
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has HQ in Hyderabad in India and is ...,59,"[0.002078805584460497, -0.02543639950454235, -...","[0.002078805584460497, -0.02543639950454235, -..."
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has HQ in Petah Tikva in Israel and is...,55,"[0.01365522388368845, -0.011359822936356068, 0...","[0.01365522388368845, -0.011359822936356068, 0..."


In [78]:
df.to_csv('companies_with_embeddings.csv', index = False)

In [99]:
df["embedding_to_arr"] = df['embedding'].apply(np.array)

In [105]:
df["embedding_to_arr_2D"] = df['embedding_to_arr'].apply(lambda x: x.reshape(-1,1))

In [106]:
df["embedding_to_arr_2D"][0].shape

(1536, 1)

In [108]:
cosine_similarity(df['embedding_to_arr_2D'][0], df['embedding_to_arr_2D'][1])


array([[ 1.,  1., -1., ..., -1.,  1., -1.],
       [-1., -1.,  1., ...,  1., -1.,  1.],
       [-1., -1.,  1., ...,  1., -1.,  1.],
       ...,
       [-1., -1.,  1., ...,  1., -1.,  1.],
       [ 1.,  1., -1., ..., -1.,  1., -1.],
       [-1., -1.,  1., ...,  1., -1.,  1.]])

In [90]:
from sklearn.metrics.pairwise import cosine_similarity
#osine_similarity = cosine_similarity(vector1, vector2)

In [136]:
def search_for_similarity(df, prompt, n=1, pprint=True):
    
    prompt_emb = np.array(get_embedding(prompt))
    
    df["similarity2"] = df['embedding_to_arr'].apply(lambda x: np.dot(x, prompt_emb))

    results = (df.sort_values("similarity2", ascending=False).head(n))
    context = results.iloc[0]['summary']
    prompt_main = f"""Only answer the question below if you are 100% certain baed on the fact.\n Context: {context}
    Q: {prompt}
    A: """
    
    return prompt_main





In [149]:
results = search_for_similarity(df, prompt = input('what is your question?'), n=1)

what is your question?tell me about Momenta


In [150]:
print(results)

Only answer the question below if you are 100% certain baed on the fact.
 Context: Momenta has HQ in Beijing in China and is in the field of Artificial intelligence. The investors of this company are Sinovation Ventures,  Tencent Holdings,  Sequoia Capital China, . More info on https://www.cbinsights.com/company/momenta
    Q: tell me about Momenta
    A: 


In [151]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "system", "content": "You are an assistant"}, 
            {"role": "user", "content":results },
    {"role": "assistant", "content": " "}], max_tokens = 100, temperature=0
)

In [152]:
response.choices[0].message.content

'Momenta is a company based in Beijing, China that specializes in artificial intelligence. It has received investments from Sinovation Ventures, Tencent Holdings, and Sequoia Capital China.'